In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
train_df  =  pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test_df = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.describe()

In [ ]:
import re
import string

def clean_review_text(text):
    text = text.lower()  # covert the text to lowercase
    text = re.sub('<.*?>','',text).strip() # remove html chars
    text = re.sub('\[|\(.*\]|\)','', text).strip() # remove text in square brackets and parenthesis
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove punctuation marks
    text = re.sub("(\\W)"," ",text).strip() # remove non-ascii chars
    text = re.sub('\S*\d\S*\s*','', text).strip()  # remove words containing numbers
    return text.strip()

train_df.comment_text = train_df.comment_text.astype(str)
train_df.comment_text = train_df.comment_text.apply(clean_review_text)
train_df.comment_text.head()

In [ ]:
import  matplotlib.pyplot as plt
import tensorflow as tf
import keras
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import roc_auc_score , accuracy_score , confusion_matrix , f1_score
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.feature_extraction.text import TfidfVectorizer
import en_core_web_sm


nlp = en_core_web_sm.load()
snow_stemmer = SnowballStemmer(language='english')
stopwords = nlp.Defaults.stop_words

In [ ]:
def apply_stemmer(text):
    words = text.split()
    sent = [snow_stemmer.stem(word) for word in words if not word in set(stopwords)]
    return ' '.join(sent)

train_df.comment_text = train_df.comment_text.apply(apply_stemmer)
train_df.comment_text.head()

In [ ]:
X = train_df.comment_text
y = train_df.drop(['id','comment_text'],axis = 1)

In [ ]:
x_train,x_test,y_train,y_test =  train_test_split(X,y,test_size = 0.2,random_state = 45)

In [ ]:
word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',     
    analyzer='word',            
    token_pattern=r'\w{1,}',    
    ngram_range=(1, 3),         
    stop_words='english',
    sublinear_tf=True)

word_vectorizer.fit(x_train)    
train_word_features = word_vectorizer.transform(x_train)

In [ ]:
X_train_transformed = word_vectorizer.transform(x_train)
X_test_transformed = word_vectorizer.transform(x_test)

In [ ]:
logistic_regression = LogisticRegression(C = 10, penalty='l2', solver = 'liblinear', random_state=100)
classifier_ovr_log = OneVsRestClassifier(logistic_regression)
classifier_ovr_log.fit(X_train_transformed, y_train)

In [ ]:
y_train_pred_proba = classifier_ovr_log.predict_proba(X_train_transformed)
y_test_pred_proba = classifier_ovr_log.predict_proba(X_test_transformed)

In [ ]:
def make_test_predictions(df,classifier):
    df.comment_text = df.comment_text.apply(clean_review_text)
    df.comment_text = df.comment_text.apply(apply_stemmer)
    X_test = df.comment_text
    X_test_transformed = word_vectorizer.transform(X_test)
    y_test_pred = classifier.predict_proba(X_test_transformed)
    return y_test_pred

y_pred=make_test_predictions(test_df,classifier_ovr_log)
y_pred_df = pd.DataFrame(y_pred,columns=y.columns)
y_pred_df

In [ ]:
submission_df = pd.concat([test_df.id, y_pred_df], axis=1)
submission_df.to_csv('submission.csv', index = False)